In [1]:
import pandas as pd

data = pd.read_csv('../뉴스 데이터/뉴스 크롤링/라벨링/실험.csv', encoding='cp949')
data.head()

,단어,개수,label
0,열린,109,1
1,최종,51,1
2,우승을,6,1
3,경기가,7,1
4,이상의,46,1


In [80]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('../뉴스 데이터/뉴스 크롤링/라벨링/실험.csv', encoding='cp949')
X = data.iloc[:, :]
y = data.iloc[:, :2]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [54]:
import re

X['단어'] = X['단어'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
X.head()

,단어,개수,label
0,열린,109,1
1,최종,51,1
2,우승을,6,1
3,경기가,7,1
4,이상의,46,1


In [55]:
from konlpy.tag import Okt
okt = Okt()

def okt_tokenizer(data):
     tokens = okt.morphs(data) # 형태소 단위로 구분 분석
     return tokens


from sklearn.feature_extraction.text import TfidfVectorizer #TF-idf 기반 벡터화

tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(X['단어'])
train_tfidf = tfidf.transform(X['단어'])

c:\Users\PC\anaconda3\envs\pandas-dev\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [56]:
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state = 0)
SA_lr.fit(train_tfidf, X['label'])

#사이킷런: 로지스틱 회귀 실습

LogisticRegression(random_state=0)

In [57]:
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 3, 3.5, 4, 4.5, 5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [58]:
SA_lr_grid_cv.fit(train_tfidf, X['label'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [59]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

{'C': 1} 0.7456


In [60]:
SA_lr_best = SA_lr_grid_cv.best_estimator_

In [61]:
test_tfidf = tfidf.transform(y['단어'])

In [62]:
test_predict = SA_lr_best.predict(test_tfidf)

In [64]:
from sklearn.metrics import accuracy_score

print('감정 분류 정확도 : ', round(accuracy_score(y['label'], test_predict), 3))

감정 분류 정확도 :  0.754


## 훈련된 세트에 새로운 단어를 넣고 긍정 부정 분류

In [75]:
# 새로운 텍스트를 직접 입력해 감성 예측 수행해봅시다!
st = input("감성을 분석할 문장을 입력하세요: ")

In [76]:
st = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(st)
print(st)

st = [" ".join(st)]
print(st)

['떨어진', '변동']
['떨어진 변동']


In [77]:
# 입력 텍스트의 벡터화
st_tfidf = tfidf.transform(st)

# 감성 분석 모델에 적용하여 예측
st_predict = SA_lr_best.predict(st_tfidf)

In [78]:
# 예측값 출력
if(st_predict ==0):
    print(st, "->> 부정 감성")
else:
    print(st, "->> 긍정 감성")

['떨어진 변동'] ->> 긍정 감성
